## K meas news catagories experiment


This note book is intended for experiment and visualization.
The dataset contains over 400 news pieces from ZingNews, Vietnamnet, Dan Tri, Thanh Nien and VTC. The pipeline is as follow:  
- Preprocess text data: we represent each of the aticles as TF-IDF form, which takes into account the frequency of words in a document and their overall importance in the corpus. By using TF-IDF, we aim to leverage the most important words in each article. 
- we applied principal component analysis (PCA) to simplify the data and remove any redundant information. In this notebook, we would reduce to 2D for better visualization
- Finally, we used k-means clustering to group the articles into different clusters

In [ ]:
!pip3 install texthero
!pip3 install spacy==3.0.0
!python3 -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached spacy-2.3.9-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.0 MB)
  Using cached thinc-7.4.6-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
  Using cached srsly-1.0.6-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (211 kB)
  Using cached catalogue-1.0.2-py2.py3-none-any.whl (16 kB)
  Attempting uninstall: srsly
    Found existing installation: srsly 2.4.5
    Uninstalling srsly-2.4.5:
      Successfully uninstalled srsly-2.4.5
  Attempting uninstall: catalogue
    Found existing installation: catalogue 2.0.8
    Uninstalling catalogue-2.0.8:
      Successfully uninstalled catalogue-2.0.8
  Attempting uninstall: thinc
    Found existing installation: thinc 8.0.17
    Uninstalling thinc-8.0.17:
      Successfully uninstalled thinc-8.0.17
  Attempting uninstall: spacy
    Found existing installation: spacy 3.0.0
    Uninstalling spacy-3

In [ ]:
import texthero
help(texthero)

Help on package texthero:

NAME
    texthero - Texthero: python toolkit for text preprocessing, representation and visualization.

PACKAGE CONTENTS
    nlp
    preprocessing
    representation
    stopwords
    visualization

DATA
    Callable = typing.Callable
    List = typing.List
    Optional = typing.Optional
    Set = typing.Set

FILE
    /usr/local/lib/python3.8/dist-packages/texthero/__init__.py




#### Text Preprocessing

In [ ]:
import pandas as pd
text="It's a pleasant   day at Bangaloré; at / (10:30) am "
series=pd.Series(text)

In [ ]:
series

0    It's a pleasant   day at Bangaloré; at / (10:3...
dtype: object

In [ ]:
import texthero as hero

hero.remove_digits(series)

0    It's a pleasant   day at Bangaloré; at / ( : )...
dtype: object

In [ ]:
#### Remove punctuations
hero.remove_punctuation(series)

0    It s a pleasant   day at Bangaloré  at    10 3...
dtype: object

In [ ]:
#### Remove Brackets
hero.remove_brackets(series)

0    It's a pleasant   day at Bangaloré; at /  am 
dtype: object

In [ ]:
hero.remove_diacritics(series)

0    It's a pleasant   day at Bangalore; at / (10:3...
dtype: object

In [ ]:
hero.remove_whitespace(series)

0    It's a pleasant day at Bangaloré; at / (10:30) am
dtype: object

In [ ]:
### Stopwords
hero.remove_stopwords(series)

0    It'  pleasant   day  Bangaloré;  / (10:30)  
dtype: object

In [ ]:
hero.clean(series)

0    pleasant day bangalore
dtype: object

In [ ]:
df = pd.read_csv(
   "https://raw.githubusercontent.com/thesunsavior/Vietnamese-Fake-News-Generation/main/data.csv"
)
df.head()

,text
0,Theo Sky News trong video nhân viên cứu hộ b...
1,Vật thể bay thứ 4 bị bắn hạ trên không phận H...
2,Ghi nhận của PV VietNamNet những ngày này ng...
3,Duy trì việc đảm bảo an toàn giao thông sau d...
4,Sau một chút chững lại MU tiếp tục thi đấu ấ...


In [ ]:
###PCA
import texthero as hero
import pandas as pd

df = pd.read_csv(
   "https://raw.githubusercontent.com/thesunsavior/Vietnamese-Fake-News-Generation/main/data.csv"
)

df['pca'] = (
   df['text']
   .pipe(hero.tfidf)###vectorizing
   .pipe(hero.pca)
)
hero.scatterplot(df, 'pca', title="PCA news")

In [ ]:
df['pca'] = (
   df['text']
   .pipe(hero.tfidf)###vectorizing
   .pipe(hero.tsne)
)
hero.scatterplot(df, 'pca', title="PCA news")

In [ ]:
# pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
df.head()

,text,pca
0,Theo Sky News trong video nhân viên cứu hộ b...,"[-0.03146625848518399, -0.10256711988459334]"
1,Vật thể bay thứ 4 bị bắn hạ trên không phận H...,"[-0.03606900530071472, -0.09662604526909242]"
2,Ghi nhận của PV VietNamNet những ngày này ng...,"[-0.04811979426719439, -0.09439816676828591]"
3,Duy trì việc đảm bảo an toàn giao thông sau d...,"[-0.05440952687360622, -0.078861870216976]"
4,Sau một chút chững lại MU tiếp tục thi đấu ấ...,"[0.9399957946708094, 0.058908962613200944]"


In [ ]:
%pip install scikit-learn==0.22.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import texthero as hero
import pandas as pd

df = pd.read_csv(
    "https://raw.githubusercontent.com/thesunsavior/Vietnamese-Fake-News-Generation/main/data.csv"
)

df['tfidf'] = (
    df['text']
    .pipe(hero.clean)
    .pipe(hero.tfidf)
)

df['pca'] = df['tfidf'].pipe(hero.tsne)

df['kmeans_labels'] = (
    df['pca']
    .pipe(hero.kmeans, n_clusters=8)
    .astype(str)
)


hero.scatterplot(df, 'pca', color='kmeans_labels', title="Vietnamese news clustering")

In [ ]:
res =df[df["kmeans_labels"] == "3"]
print(res)

                                                  text  \
7     Hà Tĩnh có lịch thi đấu thực sự thử thách khi...   
20    Người làm thuêVề nguyên tắc cơ bản mối quan h...   
38    HLV Nguyễn Huy Hoàng của SLNA chia sẻ sau trậ...   
54    Tại sân Vinh  SLNA (2 điểm) – đội vừa hoà liê...   
64    Bùi Tiến Dũng bắt chính ở 2 trận đầu của CAHN...   
90    (VTC News) -HLV Philippe Troussier sắp trở th...   
97    (VTC News) -Liên đoàn bóng đá Việt Nam (VFF) ...   
112   (VTC News) -Tại lễ xuất quân mùa giải 2023 củ...   
120   (VTC News) -Ở mùa giải 2023  mức tiền thưởng ...   
122   (VTC News) - Đây là vấn đề không ai mong muốn...   
132   (VTC News) -Hôm 30/1  phát ngôn viên Bộ Ngoại...   
147   (VTC News) -Chiều 30/1  CLB bóng đá Đông Á Th...   
162   (VTC News) - HAGL có nhiều khả năng sẽ không ...   
165   Lexus RX 2023Theo xác nhận từ đại lý  mẫu xe ...   
192   (VTC News) -Điều lệ V-League 2023 ghi rõ:  Nế...   
206    Sự thật là tôi đã không nghĩ nhiều. Đó là nh...   
218   (VTC New